# Exploring Data

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

image_width = x_train.shape[1]
image_height = x_train.shape[2]
num_channels = 1

x_train = x_train.astype('float32')
x_train = x_train.reshape(x_train.shape[0], image_height, image_width, num_channels)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_random_sample_images(x_train, n=5):
    random_indexes = np.random.choice(len(x_train), n**2, replace=False)

    for i, idx in enumerate(random_indexes):
        plt.subplot(n, n, i + 1)
        plt.axis("off")
        plt.imshow(x_train[idx])
    plt.show()

plot_random_sample_images(x_train)

# Exploring the New Activation Function: Leaky ReLU

In [ ]:
def plot_leaky_relu(negative_slope=0.2):
    x = np.linspace(-10, 10, 400)

    leaky_relu = tf.keras.layers.LeakyReLU(alpha=negative_slope)

    y = leaky_relu(x).numpy()

    plt.plot(x, y, label=f'Leaky ReLU (alpha={negative_slope})')
    plt.title('Leaky ReLU Activation Function')
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_leaky_relu(negative_slope=0.2)
plot_leaky_relu(negative_slope=0.1)

# Building Discriminator

In [ ]:
def build_discriminator(img_shape=(image_width, image_height, num_channels)):
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.Input(shape=img_shape))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding="same"))
  model.add(tf.keras.layers.LeakyReLU(alpha=0.2))

  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding="same"))
  model.add(tf.keras.layers.LeakyReLU(alpha=0.2))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dropout(rate=0.4))
  model.add(tf.keras.layers.Dense(units=1))

  return model

# Building Generator

In [ ]:
def build_generator(latent_dim=100):
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.Input(shape=(latent_dim,)))
  model.add(tf.keras.layers.Dense(units=7*7*128))
  model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
  model.add(tf.keras.layers.Reshape(target_shape=(7, 7, 128)))

  model.add(tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), strides=(2,2), padding="same"))
  model.add(tf.keras.layers.LeakyReLU(alpha=0.2))


  model.add(tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), strides=(2,2), padding="same"))
  model.add(tf.keras.layers.LeakyReLU(alpha=0.2))

  model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(7,7), padding="same", activation="tanh"))

  return model

# Training GAN

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import time
from IPython import display

In [ ]:
generator = build_generator()
discriminator = build_discriminator()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 150
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for image_batch in dataset:
            train_step(image_batch)
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, seed)
        print(f'Time for epoch {epoch + 1} is {time.time() - start} sec')
    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
x_train = (x_train - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

train(train_dataset, EPOCHS)